In [1]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
from warnings import warn
import shutil

# custom steps for mobilenetv1
from custom_steps import (
    step_mobilenet_streamline,
    step_mobilenet_convert_to_hls_layers,
    step_mobilenet_convert_to_hls_layers_separate_th,
    step_mobilenet_lower_convs,
    step_mobilenet_slr_floorplan,
)

model_file = "models/partition_1.onnx"
board = "KV260_SOM"
additional = ""

if additional != "":
    final_output_dir = "build-{}/{}-{}".format(board,model_file.split('.')[0].split('/')[1],additional)
    os.environ["FINN_BUILD_DIR"] = "/workspace/results/%s/%s-%s" %(board,model_file.split('.')[0].split('/')[1],additional)
    inter_build_dir = os.environ["FINN_BUILD_DIR"]
else:
    final_output_dir = "build-{}/{}".format(board,model_file.split('.')[0].split('/')[1])

    os.environ["FINN_BUILD_DIR"] = "/workspace/results/%s/%s" %(board,model_file.split('.')[0].split('/')[1])
    inter_build_dir = os.environ["FINN_BUILD_DIR"]

if os.path.exists(inter_build_dir):
    shutil.rmtree(inter_build_dir)
    print("Previous intermeditate build deleted!")

#Delete previous run results if exist
if os.path.exists(final_output_dir):
    shutil.rmtree(final_output_dir)
    print("Previous run results deleted!")
    
mobilenet_build_steps =  [
    step_mobilenet_streamline,
    step_mobilenet_lower_convs,
    step_mobilenet_convert_to_hls_layers_separate_th,
    "step_create_dataflow_partition",
    "step_apply_folding_config",
    "step_generate_estimate_reports",
    "step_hls_codegen",
    "step_hls_ipgen",
    "step_set_fifo_depths",
    "step_create_stitched_ip",
    "step_synthesize_bitfile",
    "step_make_pynq_driver",
    "step_deployment_package",
]

folding_config_file = "folding_config/ZCU104_folding_update.json"

cfg = build.DataflowBuildConfig(
    output_dir          = final_output_dir,
    steps               = mobilenet_build_steps,
    mvau_wwidth_max     = 80,
    target_fps          = 50,
    synth_clk_period_ns = 10.0,
    folding_config_file = folding_config_file,
    board               = board,
    shell_flow_type     = build_cfg.ShellFlowType.VIVADO_ZYNQ,
    generate_outputs=[
        build_cfg.DataflowOutputType.BITFILE,
        build_cfg.DataflowOutputType.PYNQ_DRIVER,
        build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
        build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
    ]
)

Previous intermeditate build deleted!
Previous run results deleted!


In [ ]:
build.build_dataflow_cfg(model_file, cfg)

Building dataflow accelerator from models/partition_1.onnx
Intermediate outputs will be generated in /workspace/results/KV260_SOM/partition_1
Final outputs will be generated in build-KV260_SOM/partition_1
Build log is at build-KV260_SOM/partition_1/build_dataflow.log
Running step: step_mobilenet_streamline [1/13]
Running step: step_mobilenet_lower_convs [2/13]
Running step: step_mobilenet_convert_to_hls_layers_separate_th [3/13]
Running step: step_create_dataflow_partition [4/13]
Running step: step_apply_folding_config [5/13]
Running step: step_generate_estimate_reports [6/13]
Running step: step_hls_codegen [7/13]
Running step: step_hls_ipgen [8/13]
Running step: step_set_fifo_depths [9/13]
Running step: step_create_stitched_ip [10/13]
Running step: step_synthesize_bitfile [11/13]


Traceback (most recent call last):
  File "/workspace/finn/src/finn/builder/build_dataflow.py", line 166, in build_dataflow_cfg
    model = transform_step(model, cfg)
  File "/workspace/finn/src/finn/builder/build_dataflow_steps.py", line 632, in step_synthesize_bitfile
    model = model.transform(
  File "/workspace/qonnx/src/qonnx/core/modelwrapper.py", line 140, in transform
    (transformed_model, model_was_changed) = transformation.apply(transformed_model)
  File "/workspace/finn/src/finn/transformation/fpgadataflow/make_zynq_proj.py", line 364, in apply
    model = model.transform(
  File "/workspace/qonnx/src/qonnx/core/modelwrapper.py", line 140, in transform
    (transformed_model, model_was_changed) = transformation.apply(transformed_model)
  File "/workspace/finn/src/finn/transformation/fpgadataflow/make_zynq_proj.py", line 276, in apply
    raise Exception(
Exception: Synthesis failed, no bitfile found. Check logs under /workspace/results/KV260_SOM/partition_1/vivado_zynq_p

> /workspace/finn/src/finn/transformation/fpgadataflow/make_zynq_proj.py(276)apply()
    274         )
    275         if not os.path.isfile(bitfile_name):
--> 276             raise Exception(
    277                 "Synthesis failed, no bitfile found. Check logs under %s"
    278                 % vivado_pynq_proj_dir

